In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer, default_data_collator,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
import warnings
warnings.filterwarnings('ignore')
import os
import json
import sys
sys.path.append('../../src')
from preprocessing import FinancialTweetPreprocessor
from evaluation_transformer import evaluate_transformer
from initial_balanced_dataset import create_balanced_dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tecsongacrama/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tecsongacrama/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tecsongacrama/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Create dataset (skips if dataset already exists)
create_balanced_dataset()
# Load Data
df = pd.read_csv('../../dataset/initial_balanced_tweets.csv')
print(df.shape)
# Preprocess
print("Preprocessing DataFrame...")
preprocessor = FinancialTweetPreprocessor()
df_preprocessed = preprocessor.preprocess_dataset(df, 'tweet')
pd.set_option('display.max_columns', None)
print("\nProcessed DataFrame:")
print(df_preprocessed[['processed_text', 'sentiment', 'ticker_count', 'mention_count',
                       'url_count', 'token_count', 'exclamation_count', 'question_count']].head())
print("\nAll columns in processed DataFrame:")
print(df_preprocessed.columns.tolist())
# Count zeros in every column of df_preprocessed
zero_counts = (df_preprocessed == 0).sum()
print("Number of zeros per column:")
print(zero_counts)

Both train/val and test datasets already exist. Skipping creation.
(47106, 2)
Preprocessing DataFrame...

Processed DataFrame:
                                      processed_text  sentiment  ticker_count  \
0          upholding perhaps pushing price upwards .          1             0   
1  michael k . wirth sell 52,500 share chevron co...          2             1   
2  would buy aap buy c growth . aap sell ipads ip...          1             0   
3  economic expert believe current interest rate ...          0             0   
4                    era financial boom over forever          2             0   

   mention_count  url_count  token_count  exclamation_count  question_count  
0              0          0            6                  0               0  
1              0          0           12                  0               0  
2              0          0           15                  0               0  
3              0          0           10                  0               

In [3]:
X = df_preprocessed[['processed_text', 'ticker_count', 'mention_count',
                 'url_count', 'token_count']]
y = df_preprocessed['sentiment']

# Split the Data into Training and Testing Sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")

Training set size: 41683 samples
Testing set size: 10421 samples


In [ ]:
print("\nLOADING & PREPROCESSING TEST SET")
df_test = pd.read_csv("../../dataset/test_set.csv")
# Preprocess text
preprocessor = FinancialTweetPreprocessor()
df_test = preprocessor.preprocess_dataset(df_test, text_column='tweet')
# Prepare inputs
X_test = df_test[['processed_text', 'ticker_count', 'mention_count', 'url_count', 'token_count']]
y_test = df_test['sentiment']
print(f"Testing set size: {len(X_test)} samples")

In [7]:
# ------ FinBert + QLoRA ------
# Re-Check data format and class distribution
print(f"\nDataset Configuration")
print(f"\nClass distribution in training set:")
print(y_train.value_counts().sort_index())
print(f"\nClass distribution in validation set:")
print(y_val.value_counts().sort_index())
print(f"\nClass distribution in testing set:")
print(y_test.value_counts().sort_index())


Dataset Configuration
Class distribution in training set:
sentiment
0    13895
1    13894
2    13894
Name: count, dtype: int64

Class distribution in test set:
sentiment
0    3473
1    3474
2    3474
Name: count, dtype: int64


In [8]:
print("\nLOADING FINBERT AND CONFIGURING LORA")

# Load FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# Define BitsAndBytes config for 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the base model with the quantization config
base_model = AutoModelForSequenceClassification.from_pretrained(
    "ProsusAI/finbert",
    num_labels=3,
    quantization_config=bnb_config,
    device_map="auto" # Important for QLoRA
)

base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=16,  # rank
    lora_alpha=32,
    target_modules=["query", "key", "value", "dense"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

# Apply LoRA to the base model
lora_model = get_peft_model(base_model, lora_config)
print("\nLoRA configuration applied!")
lora_model.print_trainable_parameters()


LOADING FINBERT AND CONFIGURING LORA


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
print("\nCREATING CUSTOM MODEL WITH METADATA FEATURES")

class FinBERTWithMetadata(nn.Module):
    def __init__(self, lora_model, num_features=4, num_labels=3):
        super().__init__()
        self.lora_model = lora_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + num_features, num_labels)

    def forward(self, input_ids, attention_mask, features=None):
        # Get BERT outputs
        bert_outputs = self.lora_model.base_model.model.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Get pooled output
        pooled_output = bert_outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # If features provided, concatenate them
        if features is not None:
            combined = torch.cat([pooled_output, features], dim=1)
            logits = self.classifier(combined)
        else:
            # Fallback for inference without features
            logits = self.classifier(pooled_output)

        return logits
    
# Initialize the combined model
model = FinBERTWithMetadata(lora_model)
print("Custom model with metadata features created!")

In [ ]:
print("\nPREPARING DATA WITH TEXT AND METADATA")

# Define config
feature_columns = ['ticker_count', 'mention_count', 'url_count', 'token_count']

# Scale metadata features
scaler = StandardScaler()
train_features = X_train[feature_columns].values
val_features = X_val[feature_columns].values
test_features = X_test[feature_columns].values

print("Scaling metadata features...")
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)

print("Feature statistics (after scaling):")
print(f"Mean: {train_features_scaled.mean(axis=0)}")
print(f"Std: {train_features_scaled.std(axis=0)}")

# Define tokenization function
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128
    )

print("\nTokenizing text data...")
train_encodings = tokenize_function(X_train['processed_text'].tolist())
val_encodings = tokenize_function(X_val['processed_text'].tolist())
test_encodings = tokenize_function(X_test['processed_text'].tolist())

# Define dataset class
class TextAndFeaturesDataset(Dataset):
    def __init__(self, encodings, labels, features):
        self.encodings = encodings
        self.labels = labels
        self.features = features

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        item['features'] = torch.tensor(self.features[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = TextAndFeaturesDataset(train_encodings, y_train.tolist(), train_features_scaled)
val_dataset = TextAndFeaturesDataset(val_encodings, y_val.tolist(), val_features_scaled)
test_dataset = TextAndFeaturesDataset(test_encodings, y_test.tolist(), test_features_scaled)

print(f"Training set size: {len(train_dataset)} samples")
print(f"Validation set size: {len(val_dataset)} samples")
print(f"Test set size: {len(test_dataset)} samples")

In [ ]:
print("SETTING UP CUSTOM TRAINER")

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        features = inputs.pop("features")

        # Forward pass with features
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            features=features
        )

        # Compute loss
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs.view(-1, 3), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        labels = inputs.pop("labels") if "labels" in inputs else None
        features = inputs.pop("features")

        with torch.no_grad():
            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                features=features
            )

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(outputs.view(-1, 3), labels.view(-1))

        return (loss, outputs, labels)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="../../models/finbertqlora/finbert-qlora-metadata-results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-5,
    bf16=True,
    #fp16=True if torch.cuda.is_available() else False,
    seed=42,
    logging_strategy="epoch",
    logging_steps=None,
    save_strategy="no",
    disable_tqdm=True,
    report_to="none",
    remove_unused_columns=False,
)

# Create trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

# Train
print("TRAINING MODEL")
trainer.train()

In [ ]:
# Evaluate
print("EVALUATE ON TEST SET")
evaluate_transformer(trainer, train_dataset, test_dataset, y_train, y_test, model, save_dir="../../evaluation/baseline/finbert_qlora_eval")


In [ ]:
print("SAVING FINAL MODEL AND COMPONENTS")

save_dir = "../../models/v1-1/baseline/finbert_qlora/final"
os.makedirs(save_dir, exist_ok=True)

# Save model state (includes custom classifier + LoRA)
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'feature_columns': feature_columns,
    'lora_config': lora_config,
}, os.path.join(save_dir, 'model_complete.pth'))

# Save tokenizer and LoRA adapter separately
tokenizer.save_pretrained(save_dir)
model.lora_model.save_pretrained(os.path.join(save_dir, 'qlora_adapter'))

print(f"Saved to: {save_dir}")
print("Contents:")
print("  - model_complete.pth")
print("  - tokenizer/")
print("  - lora_adapter/")
